In [1]:
from pyspark import SparkContext

In [3]:
sc.stop()

In [2]:
sc = SparkContext(appName='1_MSD_Regression')

In [3]:
# Load and check the data 
#import os.path
#baseDir = os.path.join('data')
#inputPath = os.path.join('YearPredictionMSD.csv')
#fileName = os.path.join(baseDir, inputPath)
numPartitions = 2
path = 'hdfs://localhost:9000//user/praneeth/data/YearPredictionMSD.txt'

In [7]:
rawData = sc.textFile(path, numPartitions)

In [ ]:
#Sample data
numPoints = rawData.count()
print numPoints
samplePoints = rawData.take(5)
print samplePoints

In [4]:
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.linalg import Vectors, Matrices
import numpy as np

In [5]:
def parsePoint(line):
    values = line.split(',')
    return LabeledPoint(values[0], Vectors.dense(values[1:]))

In [8]:
parsedSamplePoints = rawData.map(parsePoint).cache()
#print parsedSamplePoints.take(2)
#firstPointFeatures = parsedSamplePoints.take(1)[0].features
#firstPointLabel = parsedSamplePoints.take(1)[0].label
#print firstPointFeatures, firstPointLabel

In [9]:
parsedSamplePoints.is_cached

True

In [8]:
from pyspark.mllib.stat import Statistics,MultivariateStatisticalSummary

In [10]:
rdd = parsedSamplePoints.map(lambda a:a.features)

In [11]:
labeld = parsedSamplePoints.map(lambda a:a.label)

In [11]:
labeld.take(2)

[2001.0, 2001.0]

In [12]:
stats = Statistics.colStats(rdd)

In [13]:
print stats.count()

515345


In [19]:
print stats.mean()

[  4.33871256e+01   1.28955420e+00   8.65834709e+00   1.16412447e+00
  -6.55360070e+00  -9.52197520e+00  -2.39108942e+00  -1.79323551e+00
   3.72787581e+00   1.88238491e+00  -1.46527016e-01   2.54606336e+00
   3.37140056e+01   2.43935938e+03   1.96773420e+03   1.51485990e+03
   9.10981344e+02   8.79146721e+02   6.03737434e+02   5.17579339e+02
   3.93962215e+02   3.25733170e+02   2.88885085e+02   2.91973238e+02
   4.30319687e+01   4.33148725e+01  -4.64490057e+01  -2.76727830e+01
   1.49584906e+01   4.45148595e+01   5.13178640e+00   2.40343086e+01
   9.49881627e+00  -4.17885579e+00   4.99474661e-01   7.26522672e+01
  -5.14412560e+01   1.17921056e+02  -1.89880885e+02   2.30960821e+01
  -1.28300492e+00   1.81479733e+01  -5.19590221e+01   3.23268575e+00
  -1.48832067e+00   6.33408380e+00   7.87024269e+01   1.42696868e+02
  -8.65164764e+01   2.52407528e+01   6.37851064e+00   2.82940822e+01
   1.27722407e+01   1.70047434e+00  -1.02051553e+01   6.41012625e+01
   1.04822085e+02  -2.64808137e-02

In [20]:
print stats.numNonzeros()

[ 515345.  515345.  515345.  515345.  515345.  515345.  515345.  515345.
  515345.  515345.  515343.  515345.  515345.  515345.  515345.  515345.
  515345.  515345.  515345.  515345.  515345.  515345.  515345.  515345.
  515345.  515345.  515345.  515345.  515345.  515345.  515345.  515345.
  515345.  515345.  515345.  515345.  515345.  515345.  515345.  515345.
  515345.  515345.  515345.  515345.  515345.  515345.  515345.  515345.
  515345.  515345.  515345.  515345.  515345.  515345.  515345.  515345.
  515345.  515345.  515345.  515345.  515345.  515345.  515344.  515345.
  515345.  515345.  515345.  515345.  515345.  515345.  515345.  515345.
  515345.  515344.  515345.  515345.  515345.  515345.  515345.  515345.
  515345.  515345.  515345.  515345.  515345.  515345.  515345.  515345.
  515345.  515345.]


In [21]:
print stats.variance()

[  3.68152638e+01   2.66053259e+03   1.24387308e+03   2.66433469e+02
   5.22615510e+02   1.65321773e+02   2.12339488e+02   6.34225482e+01
   1.11996937e+02   4.26439263e+01   1.91043134e+01   6.92255624e+01
   4.95491232e+02   3.06028731e+06   1.59134388e+06   1.19427969e+06
   2.26298295e+05   3.32774375e+05   1.00805798e+05   9.57065390e+04
   4.58017789e+04   2.74562854e+04   3.49543141e+04   2.35547748e+04
   1.47451230e+04   5.12311157e+05   2.96894068e+05   4.76847915e+04
   2.66393937e+04   1.81947273e+04   9.81845540e+03   5.15666667e+03
   5.53709808e+03   2.86407320e+03   1.80543373e+03   1.16451354e+04
   1.72257455e+05   2.05570862e+05   6.79360087e+04   4.23287785e+04
   1.43599185e+04   1.43630035e+04   5.42026025e+03   1.47255448e+03
   1.72600967e+03   3.02218405e+03   2.21461908e+05   6.88380495e+04
   4.39041429e+04   1.49215056e+04   8.75266382e+03   5.63239395e+03
   4.89745921e+03   6.93533519e+03   3.34160201e+03   7.49813292e+04
   9.68051738e+04   7.11683813e+04

In [12]:
from pyspark.mllib.feature import VectorTransformer
from pyspark.mllib.feature import Normalizer

In [13]:
nor = Normalizer(1)
newrdd=nor.transform(rdd)

In [14]:
newrdd.take(2)

[DenseVector([0.0069, 0.003, 0.0101, 0.0012, -0.0024, -0.0018, -0.0035, -0.0017, 0.0011, -0.0003, 0.0005, -0.0003, 0.0014, 0.0846, 0.1316, 0.0966, 0.0566, 0.0531, 0.0452, 0.0329, 0.0348, 0.0259, 0.0139, 0.0248, -0.0012, -0.044, 0.0133, 0.0067, -0.0132, -0.0025, 0.0003, 0.0048, 0.0016, 0.0002, 0.0011, -0.0001, -0.0185, -0.0115, -0.0052, 0.0101, -0.0052, -0.0004, -0.0034, -0.0018, 0.0022, -0.0026, 0.0114, 0.0333, -0.0014, 0.0044, -0.0035, -0.0005, 0.0018, 0.0057, -0.001, -0.0029, 0.0146, 0.0089, 0.0036, -0.0062, -0.0011, 0.0011, -0.0015, -0.0132, -0.0114, -0.0049, 0.0006, 0.0098, 0.0039, 0.0008, -0.0051, -0.0057, -0.0012, 0.001, -0.0012, -0.0008, -0.0017, 0.002, -0.0075, 0.0056, 0.0018, -0.0075, 0.0082, 0.0021, 0.0002, -0.0032, 0.0095, -0.0003, -0.0038, 0.0003]),
 DenseVector([0.0044, 0.0017, 0.0064, 0.0012, -0.0009, -0.0023, 0.0008, -0.0001, 0.0017, 0.0004, 0.0002, 0.0, 0.004, 0.1875, 0.0552, 0.0417, 0.0708, 0.0379, 0.068, 0.0334, 0.029, 0.0249, 0.0129, 0.0289, 0.0018, -0.0059, 0.0148, 

In [26]:
nor1 = Normalizer(p=2)
newrdd1=nor.transform(rdd)
newrdd1.take(2)

[DenseVector([0.0069, 0.003, 0.0101, 0.0012, -0.0024, -0.0018, -0.0035, -0.0017, 0.0011, -0.0003, 0.0005, -0.0003, 0.0014, 0.0846, 0.1316, 0.0966, 0.0566, 0.0531, 0.0452, 0.0329, 0.0348, 0.0259, 0.0139, 0.0248, -0.0012, -0.044, 0.0133, 0.0067, -0.0132, -0.0025, 0.0003, 0.0048, 0.0016, 0.0002, 0.0011, -0.0001, -0.0185, -0.0115, -0.0052, 0.0101, -0.0052, -0.0004, -0.0034, -0.0018, 0.0022, -0.0026, 0.0114, 0.0333, -0.0014, 0.0044, -0.0035, -0.0005, 0.0018, 0.0057, -0.001, -0.0029, 0.0146, 0.0089, 0.0036, -0.0062, -0.0011, 0.0011, -0.0015, -0.0132, -0.0114, -0.0049, 0.0006, 0.0098, 0.0039, 0.0008, -0.0051, -0.0057, -0.0012, 0.001, -0.0012, -0.0008, -0.0017, 0.002, -0.0075, 0.0056, 0.0018, -0.0075, 0.0082, 0.0021, 0.0002, -0.0032, 0.0095, -0.0003, -0.0038, 0.0003]),
 DenseVector([0.0044, 0.0017, 0.0064, 0.0012, -0.0009, -0.0023, 0.0008, -0.0001, 0.0017, 0.0004, 0.0002, 0.0, 0.004, 0.1875, 0.0552, 0.0417, 0.0708, 0.0379, 0.068, 0.0334, 0.029, 0.0249, 0.0129, 0.0289, 0.0018, -0.0059, 0.0148, 

In [17]:
nor2 = Normalizer(float("inf"))
newrdd1=nor.transform(rdd)
newrdd1.take(2)

[DenseVector([0.0069, 0.003, 0.0101, 0.0012, -0.0024, -0.0018, -0.0035, -0.0017, 0.0011, -0.0003, 0.0005, -0.0003, 0.0014, 0.0846, 0.1316, 0.0966, 0.0566, 0.0531, 0.0452, 0.0329, 0.0348, 0.0259, 0.0139, 0.0248, -0.0012, -0.044, 0.0133, 0.0067, -0.0132, -0.0025, 0.0003, 0.0048, 0.0016, 0.0002, 0.0011, -0.0001, -0.0185, -0.0115, -0.0052, 0.0101, -0.0052, -0.0004, -0.0034, -0.0018, 0.0022, -0.0026, 0.0114, 0.0333, -0.0014, 0.0044, -0.0035, -0.0005, 0.0018, 0.0057, -0.001, -0.0029, 0.0146, 0.0089, 0.0036, -0.0062, -0.0011, 0.0011, -0.0015, -0.0132, -0.0114, -0.0049, 0.0006, 0.0098, 0.0039, 0.0008, -0.0051, -0.0057, -0.0012, 0.001, -0.0012, -0.0008, -0.0017, 0.002, -0.0075, 0.0056, 0.0018, -0.0075, 0.0082, 0.0021, 0.0002, -0.0032, 0.0095, -0.0003, -0.0038, 0.0003]),
 DenseVector([0.0044, 0.0017, 0.0064, 0.0012, -0.0009, -0.0023, 0.0008, -0.0001, 0.0017, 0.0004, 0.0002, 0.0, 0.004, 0.1875, 0.0552, 0.0417, 0.0708, 0.0379, 0.068, 0.0334, 0.029, 0.0249, 0.0129, 0.0289, 0.0018, -0.0059, 0.0148, 

In [17]:
parsedDataInit = rawData.map(parsePoint)
onlyLabels = parsedDataInit.map(lambda a: a.label)
minYear = onlyLabels.min()
maxYear = onlyLabels.max()
print maxYear, minYear

2011.0 1922.0


In [15]:
NewData = labeld.zip(newrdd)

In [22]:
NewData.first()

(2001.0,
 DenseVector([0.0069, 0.003, 0.0101, 0.0012, -0.0024, -0.0018, -0.0035, -0.0017, 0.0011, -0.0003, 0.0005, -0.0003, 0.0014, 0.0846, 0.1316, 0.0966, 0.0566, 0.0531, 0.0452, 0.0329, 0.0348, 0.0259, 0.0139, 0.0248, -0.0012, -0.044, 0.0133, 0.0067, -0.0132, -0.0025, 0.0003, 0.0048, 0.0016, 0.0002, 0.0011, -0.0001, -0.0185, -0.0115, -0.0052, 0.0101, -0.0052, -0.0004, -0.0034, -0.0018, 0.0022, -0.0026, 0.0114, 0.0333, -0.0014, 0.0044, -0.0035, -0.0005, 0.0018, 0.0057, -0.001, -0.0029, 0.0146, 0.0089, 0.0036, -0.0062, -0.0011, 0.0011, -0.0015, -0.0132, -0.0114, -0.0049, 0.0006, 0.0098, 0.0039, 0.0008, -0.0051, -0.0057, -0.0012, 0.001, -0.0012, -0.0008, -0.0017, 0.002, -0.0075, 0.0056, 0.0018, -0.0075, 0.0082, 0.0021, 0.0002, -0.0032, 0.0095, -0.0003, -0.0038, 0.0003]))

In [23]:
parsedData = NewData.map(lambda a: LabeledPoint((1 if a[0] >=1922 else 0),a[1:]))

In [24]:
weights = [.6, .2, .2]
seed = 42
parsedTrainData, parsedValData, parsedTestData = parsedData.randomSplit(weights,seed)
parsedTrainData.cache()
parsedValData.cache()
parsedTestData.cache()


PythonRDD[18] at RDD at PythonRDD.scala:43

In [26]:
sc.stop()

In [ ]:
nTrain = parsedTrainData.count()
nVal = parsedValData.count()
nTest = parsedTestData.count()

print nTrain, nVal, nTest, nTrain + nVal + nTest
print parsedData.count()

In [ ]:
model = SVMWithSGD.train(parsedTrainData, iterations=100,step=0.001,regParam=0.01)

In [ ]:
print model.threshold

In [ ]:
# Evaluating the model on training data
labelsAndPreds = parsedTrainData.map(lambda p: (p.label, float(model.predict(p.features))))
trainErr = labelsAndPreds.filter(lambda (v, p): v != p).count() / float(parsedTrainData.count())
print("Training Error = " + str(trainErr))

In [ ]:
metrics = BinaryClassificationMetrics(labelsAndPreds)
AUC = metrics.areaUnderROC
APR = metrics.areaUnderPR

print("train AreaUnderCurve = " + str(AUC))
#print("train AreaUnderPrecision" +str(APR))

In [ ]:
metrics = BinaryClassificationMetrics(labelsAndPredsval)
AUC = metrics.areaUnderROC
APR = metrics.areaUnderPR

print("validation AreaUnderCurve = " + str(AUC))
#print("validation AreaUnderPrecision" +str(APR))

metrics = BinaryClassificationMetrics(labelsAndPredstest)
AUC = metrics.areaUnderROC
APR = metrics.areaUnderPR

print("test AreaUnderCurve = " + str(AUC))
#print("test AreaUnderPrecision" +str(APR))

In [ ]:
from sklearn.metrics import confusion_matrix
import numpy as np

p = np.array(labelsAndPreds.collect())
confusion_matrix(p[:,0],p[:,1])


In [ ]:
p = np.array(labelsAndPredsval.collect())
confusion_matrix(p[:,0],p[:,1])

In [ ]:
p = np.array(labelsAndPredstest.collect())
confusion_matrix(p[:,0],p[:,1])

In [ ]:
from pyspark.mllib.classification import LogisticRegressionWithLBFGS

In [ ]:
model = LogisticRegressionWithLBFGS.train(parsedTrainData,iterations=100,regParam=0.01)

In [ ]:
model.clearThreshold

In [ ]:
# Evaluating the model on training data
labelsAndPreds = parsedTrainData.map(lambda p: (p.label, float(model.predict(p.features))))
trainErr = labelsAndPreds.filter(lambda (v, p): v != p).count() / float(parsedTrainData.count())
print("Training Error = " + str(trainErr))

In [ ]:
metrics = BinaryClassificationMetrics(labelsAndPreds)
AUC = metrics.areaUnderROC
APR = metrics.areaUnderPR

print("train AreaUnderCurve = " + str(AUC))
#print("train AreaUnderPrecision" +str(APR))

In [ ]:
p = np.array(labelsAndPreds.collect())
confusion_matrix(p[:,0],p[:,1])

In [ ]:
p = np.array(labelsAndPredsval.collect())
confusion_matrix(p[:,0],p[:,1])


In [ ]:
p = np.array(labelsAndPredstest.collect())
confusion_matrix(p[:,0],p[:,1])

In [ ]:
from pyspark.mllib.classification import LogisticRegressionWithLBFGS

In [ ]:
model = LogisticRegressionWithLBFGS.train(parsedTrainData,iterations=1000,regParam=0.01)

In [ ]:
model.clearThreshold

In [ ]:
# Evaluating the model on training data
labelsAndPreds = parsedTrainData.map(lambda p: (p.label, float(model.predict(p.features))))
trainErr = labelsAndPreds.filter(lambda (v, p): v != p).count() / float(parsedTrainData.count())
print("Training Error = " + str(trainErr))

In [ ]:
metrics = BinaryClassificationMetrics(labelsAndPreds)
AUC = metrics.areaUnderROC
APR = metrics.areaUnderPR

print("train AreaUnderCurve = " + str(AUC))
#print("train AreaUnderPrecision" +str(APR))

In [ ]:
p = np.array(labelsAndPreds.collect())
confusion_matrix(p[:,0],p[:,1])

In [ ]:
# Evaluating the model on validation data
labelsAndPredsval = parsedValData.map(lambda p: (p.label, float(model.predict(p.features))))
valErr = labelsAndPreds.filter(lambda (v, p): v != p).count() / float(parsedValData.count())
print("validation Error = " + str(valErr))

# Evaluating the model on validation data
labelsAndPredstest = parsedTestData.map(lambda p: (p.label, float(model.predict(p.features))))
testErr = labelsAndPreds.filter(lambda (v, p): v != p).count() / float(parsedTestData.count())
print("Test Error = " + str(testErr))


In [ ]:
p = np.array(labelsAndPredsval.collect())
confusion_matrix(p[:,0],p[:,1])

p = np.array(labelsAndPredstest.collect())
confusion_matrix(p[:,0],p[:,1])

In [ ]:
from pyspark.mllib.tree import GradientBoostedTrees, GradientBoostedTreesModel


In [ ]:
model = GradientBoostedTrees.trainClassifier(parsedTrainData,
    categoricalFeaturesInfo={}, numIterations=3)

In [ ]:
# Evaluate model on train instances and compute test error
predictions = model.predict(parsedTrainData.map(lambda x: x.features))
labelsAndPredictions = parsedTrainData.map(lambda lp: lp.label).zip(predictions)
testErr = labelsAndPredictions.filter(lambda (v, p): v != p).count() / float(parsedTrainData.count())
print('Test Error = ' + str(testErr))
print('Learned classification GBT model:')
print(model.toDebugString())

In [ ]:
metrics = BinaryClassificationMetrics(labelsAndPredictions)
AUC = metrics.areaUnderROC
APR = metrics.areaUnderPR

print("train AreaUnderCurve = " + str(AUC))
#print("train AreaUnderPrecision" +str(APR))


In [ ]:
p = np.array(labelsAndPredictions.collect())
confusion_matrix(p[:,0],p[:,1])

In [ ]:
# Evaluate model on val instances and compute test error
predictions = model.predict(parsedValData.map(lambda x: x.features))
labelsAndPredictions = parsedValData.map(lambda lp: lp.label).zip(predictions)
testErr = labelsAndPredictions.filter(lambda (v, p): v != p).count() / float(parsedTrainData.count())
print('validation Error = ' + str(testErr))
print('Learned classification GBT model:')
print(model.toDebugString())

In [ ]:
# Evaluate model on val instances and compute test error
predictions = model.predict(parsedTestData.map(lambda x: x.features))
labelsAndPredictions = parsedTestData.map(lambda lp: lp.label).zip(predictions)
testErr = labelsAndPredictions.filter(lambda (v, p): v != p).count() / float(parsedTestData.count())
print('Test Error = ' + str(testErr))
print('Learned classification GBT model:')
print(model.toDebugString())